# **Original Model Implementation (Modified to fit TensorFlow Version 2.14.0)**

In [ ]:
import numpy as np
import tensorflow as tf
import iris
import matplotlib.pyplot as plt
from neural_network_decision_tree import nn_decision_tree
%matplotlib inline

In [ ]:
tf.__version__

'2.14.0'

In [ ]:
np.random.seed(1943)
tf.random.set_seed(1943)

In [ ]:
x = iris.feature[:, 2:4]  # use "Petal length" and "Petal width" only
y = iris.label
d = x.shape[1]

In [ ]:
num_cut = [1, 1]  # "Petal length" and "Petal width"
num_leaf = np.prod(np.array(num_cut) + 1)
num_class = 3

In [ ]:
sess = tf.compat.v1.InteractiveSession()

In [ ]:
# x_ph = tf.compat.v1.placeholder(tf.float32, [None, d])
# y_ph = tf.placeholder(tf.float32, [None, num_class])
x_ph = tf.Variable(initial_value=tf.zeros((1, d), dtype=tf.float32))
y_ph = tf.Variable(initial_value=tf.zeros((1, num_class), dtype=tf.float32))

In [ ]:
cut_points_list = [tf.Variable(tf.random.uniform([i])) for i in num_cut]
leaf_score = tf.Variable(tf.random.uniform([num_leaf, num_class]))

In [ ]:
y_pred = nn_decision_tree(x_ph, cut_points_list, leaf_score, temperature=0.1)
# loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=y_pred, onehot_labels=y_ph))
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
loss = loss_function(y_ph, y_pred)
loss = tf.reduce_mean(loss)

In [ ]:
# opt = tf.train.AdamOptimizer(0.1)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
# train_step = opt.minimize(loss)
@tf.function
def train_step(x, y, cut_points_list, leaf_score):
    with tf.GradientTape() as tape:
        y_pred = nn_decision_tree(x, cut_points_list, leaf_score, temperature=0.1)
        print("Shapes - y_pred:", y_pred.shape, "y:", y.shape)  # Print shapes for debugging
        y_pred = tf.reshape(y_pred, [-1, num_class])  # Reshape y_pred to match the shape of y
        current_loss = loss_function(y, y_pred)

    gradients = tape.gradient(current_loss, [x_ph] + cut_points_list + [leaf_score])
    optimizer.apply_gradients(zip(gradients, [x_ph] + cut_points_list + [leaf_score]))
    return current_loss

In [ ]:
# Convert numpy arrays to Tensor
x = tf.constant(x, dtype=tf.float32)
# y_oneHot = tf.one_hot(y, num_class)

# Your training loop
num_epochs = 1000  # Set the number of epochs
for epoch in range(num_epochs):
    current_loss = train_step(x, y, cut_points_list, leaf_score)
    if epoch % 200 == 0:
        print(current_loss.numpy())

y_pred = nn_decision_tree(x, cut_points_list, leaf_score, temperature=0.1)
error_rate = 1 - np.mean(np.argmax(y_pred, axis=1) == np.argmax(y, axis=1))
print(f'error rate {error_rate:.2f}')

Values - y_pred: Tensor("MatMul_2:0", shape=(150, 3), dtype=float32) y: Tensor("y:0", shape=(150, 3), dtype=int64)
Shapes - y_pred: (150, 3) y: (150, 3)


Values - y_pred: Tensor("MatMul_2:0", shape=(150, 3), dtype=float32) y: Tensor("y:0", shape=(150, 3), dtype=int64)
Shapes - y_pred: (150, 3) y: (150, 3)
1.2471884
0.12529777
0.12489141
0.12326012
0.121581145
error rate 0.03
